In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# List of file names
file_names = [
    "010224_ALL_VAX.xlsx", "010924_ALL_VAX.xlsx", "011624_ALL_VAX.xlsx",
    "012224_ALL_VAX.xlsx", "012924_ALL_VAX.xlsx", "020524_ALL_VAX.xlsx",
    "021224_ALL_VAX.xlsx", "022024_ALL_VAX.xlsx", "022624_ALL_VAX.xlsx",
    "030424_ALL_VAX.xlsx", "031124_ALL_VAX.xlsx", "031824_ALL_VAX.xlsx",
    "032524_ALL_VAX.xlsx", "040124_ALL_VAX.xlsx", "040824_ALL_VAX.xlsx",
    "041524_ALL_VAX.xlsx", "042224_ALL_VAX.xlsx", "042924_ALL_VAX.xlsx",
    "ALL_VAX_2024-05-08.xlsx", "ALL_VAX_2024-05-13.xlsx", "ALL_VAX_2024-05-20.xlsx",
    "ALL_VAX_2024-05-27.xlsx"
]

# Load all weeks into a list
weeks = [pd.read_excel(file) for file in file_names]

# Fill NaN values with 0
for week in weeks:
    week.fillna(0, inplace=True)

week_dates = [
    '01-02-24', '01-09-24', '01-16-24', '01-22-24', '01-29-24', '02-05-24',
    '02-12-24', '02-20-24', '02-26-24', '03-04-24', '03-11-24', '03-18-24',
    '03-25-24', '04-01-24', '04-08-24', '04-15-24', '04-22-24', '04-29-24',
    '05-06-24', '05-13-24', '05-20-24', '05-27-24'
]

# Function to process weeks data
def process_weeks(weeks, week_dates, column_name):
    top_15_list = []
    top_16_to_100_list = []

    for week, date in zip(weeks, week_dates):
        week_copy = week[['PROVIDER_NAME', column_name]].copy()
        
        # Top 15
        top_15 = week_copy.sort_values(by=column_name, ascending=False).head(15)
        top_15['Week'] = date
        top_15.reset_index(drop=True, inplace=True)
        top_15_list.append(top_15)
        
        # Top 16-100
        top_16_to_100 = week_copy.sort_values(by=column_name, ascending=False).iloc[15:]
        top_16_to_100['Week'] = date
        top_16_to_100.reset_index(drop=True, inplace=True)
        top_16_to_100_list.append(top_16_to_100)

    return pd.concat(top_15_list, ignore_index=True), pd.concat(top_16_to_100_list, ignore_index=True)

# Process for different columns
columns_to_process = ['HL7_TOTAL_COUNT_8_DAYS', 'HL7_ETHNICITY_COUNT_8_DAYS', 'UNKNOWN_GENDER', 'SHOT_ADMIN_DATE_ON_DOB']

for column in columns_to_process:
    top_15_df, top_16_to_100_df = process_weeks(weeks, week_dates, column)

    # Left join on top 15 providers with 16-100 to fill the gaps when the top 15 is empty
    top15_left_df = pd.merge(top_15_df, top_16_to_100_df, on='PROVIDER_NAME', how='left')
    
    top15_left_hl7 = pd.melt(
        top15_left_df,
        id_vars=['PROVIDER_NAME'],
        value_vars=[f'{column}_x', f'{column}_y'],
        var_name=column,
        value_name=f'{column}_value'
    )
    
    # Melt the DataFrame to combine 'Week_x' and 'Week_y' into one column called 'Week'
    top_15_left_week = pd.melt(
        top15_left_df,
        id_vars=['PROVIDER_NAME'],
        value_vars=['Week_x', 'Week_y'],
        var_name='Week',
        value_name='Week_value'
    )
    
    # Replace 'HL7_TOTAL_8DAYS' with 'HL7_TOTAL_8DAYS_value'
    top15_left_hl7[column] = top15_left_hl7[f'{column}_value']
    top15_left_hl7.drop(columns=[f'{column}_value'], inplace=True)
    
    # Replace 'Week' values with 'Week_value'
    top_15_left_week['Week'] = top_15_left_week['Week_value']
    top_15_left_week.drop(columns=['Week_value'], inplace=True)
    
    # Concatenate two dataframes
    tmp_df = pd.concat([top15_left_hl7[['PROVIDER_NAME', column]], top_15_left_week['Week']], axis=1)
    tmp_df_nodup = tmp_df.drop_duplicates()
    
    # Save to Excel
    tmp_df_nodup.to_excel(f"top15_{column}_052924-super-consolidated_code.xlsx", index=False)
